In [1]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup as bs

os.chdir("/home/ubuntu/real_estate_dashboard")
print(os.getcwd())

df = pd.read_csv("./0_data/legal_info_b.csv")
df

/home/ubuntu/real_estate_dashboard


,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자,법정동시군구코드,법정동읍면동코드,주소
0,1111010100,서울특별시,종로구,청운동,NaN,19880423,,11110,10100,서울특별시 종로구 청운동
1,1111010200,서울특별시,종로구,신교동,NaN,19880423,,11110,10200,서울특별시 종로구 신교동
2,1111010300,서울특별시,종로구,궁정동,NaN,19880423,,11110,10300,서울특별시 종로구 궁정동
3,1111010400,서울특별시,종로구,효자동,NaN,19880423,,11110,10400,서울특별시 종로구 효자동
4,1111010500,서울특별시,종로구,창성동,NaN,19880423,,11110,10500,서울특별시 종로구 창성동
...,...,...,...,...,...,...,...,...,...,...
20273,5013032022,제주특별자치도,서귀포시,표선면,하천리,20060701,,50130,32022,제주특별자치도 서귀포시 표선면 하천리
20274,5013032023,제주특별자치도,서귀포시,표선면,성읍리,20060701,,50130,32023,제주특별자치도 서귀포시 표선면 성읍리
20275,5013032024,제주특별자치도,서귀포시,표선면,가시리,20060701,,50130,32024,제주특별자치도 서귀포시 표선면 가시리
20276,5013032025,제주특별자치도,서귀포시,표선면,세화리,20060701,,50130,32025,제주특별자치도 서귀포시 표선면 세화리


In [2]:
# 생성한 법정코드 고윳값 찾기
LAWD_CD_list = df['법정동시군구코드'].unique()
print(len(LAWD_CD_list))
LAWD_CD_list

250


array([11110, 11140, 11170, 11200, 11215, 11230, 11260, 11290, 11305,
       11320, 11350, 11380, 11410, 11440, 11470, 11500, 11530, 11545,
       11560, 11590, 11620, 11650, 11680, 11710, 11740, 26110, 26140,
       26170, 26200, 26230, 26260, 26290, 26320, 26350, 26380, 26410,
       26440, 26470, 26500, 26530, 26710, 27110, 27140, 27170, 27200,
       27230, 27260, 27290, 27710, 28110, 28140, 28177, 28185, 28200,
       28237, 28245, 28260, 28710, 28720, 29110, 29140, 29155, 29170,
       29200, 30110, 30140, 30170, 30200, 30230, 31110, 31140, 31170,
       31200, 31710, 36110, 41111, 41113, 41115, 41117, 41131, 41133,
       41135, 41150, 41171, 41173, 41190, 41210, 41220, 41250, 41271,
       41273, 41281, 41285, 41287, 41290, 41310, 41360, 41370, 41390,
       41410, 41430, 41450, 41461, 41463, 41465, 41480, 41500, 41550,
       41570, 41590, 41610, 41630, 41650, 41670, 41800, 41820, 41830,
       42110, 42130, 42150, 42170, 42190, 42210, 42230, 42720, 42730,
       42750, 42760,

In [3]:
import time

# 실행 전 시간 기록
start_time = time.time()

# API 키 세팅
api_key = 'odgWuZqEkLq3B8i3URl%2BvXJl42Lr61W8sPd0edRlzlvelJmFSPtJ%2B%2BpdQXpU8lhVdGN1yvYOHsuRJNH8G3m6aQ%3D%3D'

# 컬럼
columns = ['갱신요구권사용',
'건축년도',
'계약구분',
'계약기간',
'년',
'단지',
'법정동',
'보증금',
'시군구',
'월',
'월세',
'일',
'전용면적',
'종전계약보증금',
'종전계약월세',
'지번',
'지역코드',
'층']

# 서비스 url
url = "http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcOffiRent"

# 에러 추출용
error = []

# 데이터 저장용
total = pd.DataFrame()

for i in range(len(LAWD_CD_list)):
    
    # 데이터를 담는 리스트
    df_total = []
    
    # 추출하는 연월
    year_month_key = "202101"

    # 요청변수 파라미터 설정
    params = "?" + \
             "serviceKey=" + api_key + "&" + \
             "LAWD_CD=" + str(LAWD_CD_list[i]) + "&" + \
             "DEAL_YMD=" + year_month_key
    
    res = requests.get(url + params)
    soup = bs(res.text, 'xml')
    items = soup.find_all('item')
    
    if(len(items) > 0):
        # 수집된 데이터를 데이터 프레임 형식으로 변환
        for j in range(len(items)):
            df_raw = []
            for column in columns:
                # 컬럼 데이터가 없는 경우 처리
                try:
                    df_raw.append(items[j].find(column).text)
                except:
                    print(LAWD_CD_list[i], j , column) # 에러 컬럼 파악 여부
                    df_raw.append('없음')
            # 데이터 행 하나를 더해줍니다        
            df_total.append(df_raw)
            
    else:
        # 해당 지역에 거래 데이터가 없는 경우
        error.append(LAWD_CD_list[i])
        continue
        
    df = pd.DataFrame(df_total)
    df.columns = columns
    
    total = pd.concat([total,df])
    total.columns = columns
    
    print(f"{LAWD_CD_list[i]} 데이터 추출 완료")
        
        
        
        
df = pd.read_csv("./0_data/legal_info_b.csv")      

for i in range(len(error)):
     print(f'{error[i]} 거래 데이터가 없습니다')
    
# 실행 후 시간 기록
end_time = time.time()    
    
# 실행 시간 계산
execution_time = end_time - start_time
print(f"실행 시간: {execution_time} 초")   
    
total.reset_index(drop=True,inplace=True)
total

11110 데이터 추출 완료
11140 데이터 추출 완료
11170 79 건축년도
11170 86 건축년도
11170 87 건축년도
11170 88 건축년도
11170 91 건축년도
11170 92 건축년도
11170 데이터 추출 완료
11200 82 건축년도
11200 102 건축년도
11200 데이터 추출 완료
11215 23 건축년도
11215 27 건축년도
11215 34 건축년도
11215 데이터 추출 완료
11230 20 건축년도
11230 21 건축년도
11230 24 건축년도
11230 28 건축년도
11230 31 건축년도
11230 33 건축년도
11230 39 건축년도
11230 48 건축년도
11230 52 건축년도
11230 78 건축년도
11230 227 건축년도
11230 데이터 추출 완료
11260 데이터 추출 완료
11290 19 건축년도
11290 데이터 추출 완료
11305 데이터 추출 완료
11320 49 건축년도
11320 51 건축년도
11320 52 건축년도
11320 54 건축년도
11320 57 건축년도
11320 58 건축년도
11320 데이터 추출 완료
11350 데이터 추출 완료
11380 54 건축년도
11380 56 건축년도
11380 81 건축년도
11380 84 건축년도
11380 데이터 추출 완료
11410 75 건축년도
11410 88 건축년도
11410 138 건축년도
11410 데이터 추출 완료
11440 74 건축년도
11440 76 건축년도
11440 80 건축년도
11440 85 건축년도
11440 89 건축년도
11440 90 건축년도
11440 92 건축년도
11440 95 건축년도
11440 97 건축년도
11440 100 건축년도
11440 106 건축년도
11440 108 건축년도
11440 169 건축년도
11440 데이터 추출 완료
11470 데이터 추출 완료
11500 41 건축년도
11500 51 건축년도
11500 63 건축년도
11500 96 건축년도
11500 178 건

41463 0 건축년도
41463 1 건축년도
41463 62 건축년도
41463 데이터 추출 완료
41465 26 건축년도
41465 27 건축년도
41465 28 건축년도
41465 30 건축년도
41465 31 건축년도
41465 33 건축년도
41465 34 건축년도
41465 35 건축년도
41465 37 건축년도
41465 39 건축년도
41465 40 건축년도
41465 42 건축년도
41465 43 건축년도
41465 44 건축년도
41465 45 건축년도
41465 46 건축년도
41465 47 건축년도
41465 48 건축년도
41465 49 건축년도
41465 50 건축년도
41465 51 건축년도
41465 52 건축년도
41465 88 건축년도
41465 데이터 추출 완료
41480 15 건축년도
41480 29 건축년도
41480 48 건축년도
41480 65 건축년도
41480 데이터 추출 완료
41500 5 건축년도
41500 9 건축년도
41500 10 건축년도
41500 데이터 추출 완료
41550 데이터 추출 완료
41570 115 건축년도
41570 130 건축년도
41570 144 건축년도
41570 147 건축년도
41570 148 건축년도
41570 182 건축년도
41570 191 건축년도
41570 데이터 추출 완료
41590 315 건축년도
41590 321 건축년도
41590 355 건축년도
41590 356 건축년도
41590 357 건축년도
41590 376 건축년도
41590 데이터 추출 완료
41630 데이터 추출 완료
41650 데이터 추출 완료
41670 데이터 추출 완료
41830 데이터 추출 완료
43111 데이터 추출 완료
43112 데이터 추출 완료
43113 데이터 추출 완료
43114 8 건축년도
43114 9 건축년도
43114 10 건축년도
43114 11 건축년도
43114 데이터 추출 완료
43130 데이터 추출 완료
43770 데이터 추출 완료
44131 데이터 추출 완료
44133

,갱신요구권사용,건축년도,계약구분,계약기간,년,단지,법정동,보증금,시군구,월,월세,일,전용면적,종전계약보증금,종전계약월세,지번,지역코드,층
0,,1998,,,2021,파크뷰타워,필운동,500,종로구,1,50,23,21.85,,,285-5,11110,10
1,,2008,,,2021,광화문 스페이스본,사직동,"28,300",종로구,1,0,7,39.54,,,9,11110,6
2,,2008,,,2021,광화문 스페이스본,사직동,"32,000",종로구,1,0,23,39.54,,,9,11110,9
3,,2004,,,2021,용비어천가,내수동,"26,000",종로구,1,0,4,41.34,,,75,11110,5
4,,2004,,,2021,광화문시대,내수동,"20,000",종로구,1,20,12,33.3,,,74,11110,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14969,,없음,,,2021,더원,강정동,300,서귀포시,1,33,23,26.93,,,157,50130,5
14970,,2018,,,2021,(155-4),강정동,"10,000",서귀포시,1,25,30,61.24,,,155-4,50130,9
14971,,2017,,,2021,빌라드 그리움,대정읍 상모리,200,서귀포시,1,57,30,56.79,,,3841,50130,3
14972,,2017,,,2021,남영에듀클래스,대정읍 보성리,300,서귀포시,1,45,26,24.89,,,2476-10,50130,3


In [4]:
directory = '0_data/elementary_school'

if not os.path.exists(directory):
    os.makedirs(directory)

file_name = f"{directory}/elementary_school_{str(year_month_key)}.csv"

total.to_csv(file_name, index = False)